In [1]:
!prepare_transcripts -g Saccharomyces_cerevisiae.R64-1-1.112.gtf -f Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa -o RiboCode_annot

[2024-07-02 20:18:22] Preparing annotation files ...
	Reading the GTF file: Saccharomyces_cerevisiae.R64-1-1.112.gtf .......
	Process the transcripts ....
	Saving the transcripts.pickle
[2024-07-02 20:18:23] The step of preparing transcript annotation has been completed.


In [3]:
!OutputTranscriptInfo -c RiboCode_annot/transcripts_cds.txt -g Saccharomyces_cerevisiae.R64-1-1.112.gtf -f RiboCode_annot/transcripts_sequence.fa -o longest.transcripts.info.txt -O all.transcripts.info.txt

Starting outputing longest trans...
6600  transcripts will be used in the follow analysis.

Finishing!
Starting outputing all trans...
Finishing!


In [4]:
!GetProteinCodingSequence -i RiboCode_annot/transcripts_sequence.fa  -c longest.transcripts.info.txt -o output_prefix --mode whole --table 1

6600  transcripts will be used in the follow analysis.

Notes: There are 0 transcripts whose cds sequence cannot be divided by 3!
Finish the step of extracting sequences!


In [7]:
!fastqc SRR5008134.fastq -o before_trim/

null
Started analysis of SRR5008134.fastq
Approx 5% complete for SRR5008134.fastq
Approx 10% complete for SRR5008134.fastq
Approx 15% complete for SRR5008134.fastq
Approx 20% complete for SRR5008134.fastq
Approx 25% complete for SRR5008134.fastq
Approx 30% complete for SRR5008134.fastq
Approx 35% complete for SRR5008134.fastq
Approx 40% complete for SRR5008134.fastq
Approx 45% complete for SRR5008134.fastq
Approx 50% complete for SRR5008134.fastq
Approx 55% complete for SRR5008134.fastq
Approx 60% complete for SRR5008134.fastq
Approx 65% complete for SRR5008134.fastq
Approx 70% complete for SRR5008134.fastq
Approx 75% complete for SRR5008134.fastq
Approx 80% complete for SRR5008134.fastq
Approx 85% complete for SRR5008134.fastq
Approx 90% complete for SRR5008134.fastq
Approx 95% complete for SRR5008134.fastq
Analysis complete for SRR5008134.fastq


In [8]:
!cutadapt -m 15 -M 35 --match-read-wildcards -a CTGTAGGCACCATCAAT -o SRR5008134.trimmed.fastq SRR5008134.fastq > SRR5008134_trimmed.log

Done           00:08:36    54,157,010 reads @   9.5 µs/read;   6.29 M reads/minute


In [9]:
!fastq_quality_filter -Q33 -v -q 25 -p 75 -i SRR5008134.trimmed.fastq -o SRR5008134.trimmed.Qfilter.fastq > SRR5008134.Qfilter.log

In [11]:
!bowtie-build Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa bowtie_index/bowtie_index

Settings:
  Output files: "bowtie_index.*.ebwt"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 5 (one in 32)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 3039276
Using parameters --bmax 2279457 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 2279457 --dcv 1024
Constructing

In [13]:
# bowtie_noncoding_index=/Share2/home/lifj/Reference/yeast/saccharomyces/Bowtie_noncoding_index/saccharomyces_noncoding
!bowtie -n 0 -y -a --norc --best --strata -S -p 4 -l 15 --un=noncontam_SRR5008134.fastq bowtie_index/bowtie_index -q SRR5008134.trimmed.Qfilter.fastq SRR5008134.alin

Setting the index via positional argument will be deprecated in a future release. Please use -x option instead.
# reads processed: 53438435
# reads with at least one alignment: 14453944 (27.05%)
# reads that failed to align: 38984491 (72.95%)
Reported 22960671 alignments


In [14]:
!STAR --runThreadN 8 \
--runMode genomeGenerate \
--genomeDir STAR_genome_index/ \
--genomeFastaFiles Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa \
--sjdbGTFfile Saccharomyces_cerevisiae.R64-1-1.112.gtf \
--sjdbOverhang 100

	/home/zzz0054/miniconda3/envs/ribo4/bin/STAR-avx2 --runThreadN 8 --runMode genomeGenerate --genomeDir STAR_genome_index/ --genomeFastaFiles Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa --sjdbGTFfile Saccharomyces_cerevisiae.R64-1-1.112.gtf --sjdbOverhang 100
	STAR version: 2.7.11b   compiled: 2024-03-19T08:38:59+0000 :/opt/conda/conda-bld/star_1710837244939/work/source
Jul 02 20:53:31 ..... started STAR run
Jul 02 20:53:31 ... starting to generate Genome files
Jul 02 20:53:32 ..... processing annotations GTF
!!!!! WARNING: --genomeSAindexNbases 14 is too large for the genome size=12157105, which may cause seg-fault at the mapping step. Re-run genome generation with recommended --genomeSAindexNbases 10
Jul 02 20:53:32 ... starting to sort Suffix Array. This may take a long time...
Jul 02 20:53:32 ... sorting Suffix Array chunks and saving them to disk...
Jul 02 20:53:34 ... loading chunks from disk, packing SA...
Jul 02 20:53:34 ... finished generating suffix array
Jul 02 20:53:34 

In [15]:
!STAR --runThreadN 8 --outFilterType Normal --outWigType wiggle --outWigStrand Stranded --outWigNorm RPM --alignEndsType EndToEnd --outFilterMismatchNmax 1 --outFilterMultimapNmax 1 --genomeDir STAR_genome_index/ --readFilesIn noncontam_SRR5008134.fastq --outFileNamePrefix  ./ --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --outSAMattributes All

	/home/zzz0054/miniconda3/envs/ribo4/bin/STAR-avx2 --runThreadN 8 --outFilterType Normal --outWigType wiggle --outWigStrand Stranded --outWigNorm RPM --alignEndsType EndToEnd --outFilterMismatchNmax 1 --outFilterMultimapNmax 1 --genomeDir STAR_genome_index/ --readFilesIn noncontam_SRR5008134.fastq --outFileNamePrefix ./ --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --outSAMattributes All
	STAR version: 2.7.11b   compiled: 2024-03-19T08:38:59+0000 :/opt/conda/conda-bld/star_1710837244939/work/source
Jul 02 20:54:51 ..... started STAR run
Jul 02 20:54:51 ..... loading genome
Jul 02 20:54:52 ..... started mapping
Jul 02 21:01:05 ..... finished mapping
Jul 02 21:01:05 ..... started sorting BAM
Jul 02 21:01:13 ..... started wiggle output
Jul 02 21:01:34 ..... finished successfully


In [17]:
!samtools sort -T Aligned.toTranscriptome.out.sorted -o Aligned.toTranscriptome.out.sorted.bam Aligned.toTranscriptome.out.bam

[bam_sort_core] merging from 4 files and 1 in-memory blocks...


In [18]:
!samtools index Aligned.toTranscriptome.out.sorted.bam ## mapped to transcriptome

In [19]:
!samtools index Aligned.sortedByCoord.out.bam ## mapped to genome

In [25]:
!metaplots -a RiboCode_annot -r Aligned.toTranscriptome.out.sorted.bam -o metaplots/

[2024-07-02 21:31:22] Create metaplot file and predict the P-site locations ...
	Loading transcripts.pickle ...
[2024-07-02 21:32:06] Complete prediction of the P-site locations


In [24]:
!Periodicity -i Aligned.toTranscriptome.out.sorted.bam -a RiboCode_annot -o ./periodicity/ -c longest.transcripts.info.txt -L 25 -R 35

	Loading transcripts.pickle ...
6600  transcripts will be used in the follow analysis.

There are 6600 transcripts with both start and stop codon will be used for following analysis.
Start calculating read density...
There are 6600 used for statistics.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont:

In [28]:
!RiboDensityOfDiffFrames -f attributes.txt -c longest.transcripts.info.txt -o DiffFrames  --plot yes

Traceback (most recent call last):
  File "/home/zzz0054/miniconda3/envs/ribo4/bin/RiboDensityOfDiffFrames", line 8, in <module>
    sys.exit(main())
  File "/home/zzz0054/miniconda3/envs/ribo4/lib/python3.8/site-packages/RiboMiner/RiboDensityOfDiffFrames.py", line 165, in main
    parse_args_for_frame_density()
  File "/home/zzz0054/miniconda3/envs/ribo4/lib/python3.8/site-packages/RiboMiner/RiboDensityOfDiffFrames.py", line 113, in parse_args_for_frame_density
    bamFiles,readLengths,Offsets,bamLegends=parse_bamListFile(options.bamListFile)
  File "/home/zzz0054/miniconda3/envs/ribo4/lib/python3.8/site-packages/RiboMiner/FunctionDefinition.py", line 394, in parse_bamListFile
    readLengths=line.strip().split("\t")[1]
IndexError: list index out of range


In [1]:
!LengthDistribution -i SRR5008134.trimmed.Qfilter.fastq -o SRR5008134 -f fastq

Start the step of length statistics...
The total reads number is: 53431688
The zero length number is: 0
The mean of reads length is: 26.875370903498315
The sd of reads length is: 3.4577385275217702
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
Finish the step of length statistics!


In [3]:
!StatisticReadsOnDNAsContam -i  Aligned.sortedByCoord.out.bam  -g Saccharomyces_cerevisiae.R64-1-1.112.gtf -o  StatisticReadsOnDNAsContam

1000000 BAM alignments records processed.

2000000 BAM alignments records processed.

3000000 BAM alignments records processed.

4000000 BAM alignments records processed.

5000000 BAM alignments records processed.

6000000 BAM alignments records processed.

7000000 BAM alignments records processed.

8000000 BAM alignments records processed.

9000000 BAM alignments records processed.

10000000 BAM alignments records processed.

11000000 BAM alignments records processed.

12000000 BAM alignments records processed.

13000000 BAM alignments records processed.

14000000 BAM alignments records processed.

15000000 BAM alignments records processed.

16000000 BAM alignments records processed.

17000000 BAM alignments records processed.

18000000 BAM alignments records processed.

19000000 BAM alignments records processed.

20000000 BAM alignments records processed.

Start the step of plot...
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family

In [ ]:
!MetageneAnalysisForTheWholeRegions -f attributes.txt -c longest.transcripts.info.txt -o MetageneAnalysisForTheWholeRegions -b 15,90,15 -l 100 -n 10 -m 1 -e 5